In [1]:
# Could we implement a quantile regression strategy to see if the factor portfolios explain more of the higher/lower parts of return dist?


import statsmodels.formula.api as smf
from fama_data.FamaData import FamaData
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
port = pd.read_csv('http://diether.org/prephd/10-port_mom_ew.csv',parse_dates=['caldt'])
fac = pd.read_csv('http://diether.org/prephd/10-factor.csv',parse_dates=['caldt'])
port = port.merge(fac,on='caldt',how='inner')
port=port.query('caldt >= "1963-07-31" and caldt <= "2023-02-28"')
names = ['exp0','exp1','exp2','exp3','exp4']
port[names] = port[[s[2:] for s in names]].sub(port['rf'],axis='index')
port.head()

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,rf,exp0,exp1,exp2,exp3,exp4
438,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,0.27,-1.913322,-1.554810,-0.782732,-1.249859,-0.655561
439,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,0.25,2.511947,3.155911,4.023330,4.151903,6.251132
440,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,0.27,-2.887577,-0.995381,-1.424537,-1.428321,-1.958062
441,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,0.29,0.032188,0.420325,0.222692,1.612940,3.047489
442,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,0.27,-2.295639,-1.325293,-1.147067,-1.444566,-1.542383


In [3]:
port['spread'] = port['p4'] - port['p0']
if_df = pd.read_csv('exported/lasso_industry_factors.csv')
ff_df = pd.read_csv('exported/alt_ff_factors.csv')
if_df

,AgricMinusSteel,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusChips,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks,caldt
0,0.606461,0.584298,-0.162165,0.134815,-0.584478,0.766064,0.173176,-0.115244,0.310362,-0.056372,0.146697,0.420152,1963-07-31
1,-1.229077,-1.070887,0.144983,0.303882,0.321849,0.601337,-0.134279,0.163655,0.445749,0.254621,0.827600,1.035070,1963-08-31
2,-0.343146,0.259836,0.005897,0.552625,-0.028619,0.790125,0.420801,0.161331,0.375940,0.036411,-0.987600,-0.225803,1963-09-30
3,-0.271164,-1.422507,-0.698224,0.371898,-0.230750,0.284838,-0.902084,0.212463,1.181920,-0.068399,-0.953473,0.763498,1963-10-31
4,-0.204720,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,-0.305456,0.028854,0.669985,-0.169772,-0.136064,0.241690,1963-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2.090395,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.596291,-0.110595,-1.796609,-0.149154,0.733346,-0.270419,2022-05-31
707,1.153246,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.462822,1.383838,2.087324,0.563895,1.437000,2.487985,2022-06-30
708,-1.994577,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-2.691967,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644,2022-07-31
709,0.862550,1.557684,1.101197,0.781935,1.645477,-0.039063,0.606936,1.632523,0.272284,0.137784,-0.035310,-0.413964,2022-08-31


In [8]:
df = pd.concat([ff_df.drop('caldt', axis = 1), if_df.drop('caldt', axis = 1)], axis=1)
df = df.drop('HshldMinusChips', axis = 1)

In [9]:
regressors = df.columns
df = pd.concat([port.reset_index(drop = True), df.reset_index(drop = True)], axis=1)
df

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2022-05-31,-5.392614,-2.553349,0.502323,-0.183222,1.199527,-0.34,-1.85,8.41,2.48,...,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.110595,-1.796609,-0.149154,0.733346,-0.270419
707,2022-06-30,-5.344633,-8.743887,-8.291228,-4.778448,-9.131728,-8.43,2.09,-5.97,0.79,...,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.383838,2.087324,0.563895,1.437000,2.487985
708,2022-07-29,11.776372,11.033183,9.284861,6.875124,8.193744,9.57,2.81,-4.10,-3.96,...,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644
709,2022-08-31,0.317134,-2.100848,-2.140245,-2.222316,0.123339,-3.77,1.39,0.31,2.10,...,1.557684,1.101197,0.781935,1.645477,-0.039063,1.632523,0.272284,0.137784,-0.035310,-0.413964


In [10]:
names = ['exp0','exp1','exp2','exp3','exp4']
df[names] = df[[s[2:] for s in names]].sub(df['rf'],axis='index')
df.head()

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690


In [11]:
df['spread']=df['exp4']-df['exp0']
df

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2022-05-31,-5.392614,-2.553349,0.502323,-0.183222,1.199527,-0.34,-1.85,8.41,2.48,...,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.110595,-1.796609,-0.149154,0.733346,-0.270419
707,2022-06-30,-5.344633,-8.743887,-8.291228,-4.778448,-9.131728,-8.43,2.09,-5.97,0.79,...,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.383838,2.087324,0.563895,1.437000,2.487985
708,2022-07-29,11.776372,11.033183,9.284861,6.875124,8.193744,9.57,2.81,-4.10,-3.96,...,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644
709,2022-08-31,0.317134,-2.100848,-2.140245,-2.222316,0.123339,-3.77,1.39,0.31,2.10,...,1.557684,1.101197,0.781935,1.645477,-0.039063,1.632523,0.272284,0.137784,-0.035310,-0.413964


In [12]:
fd = FamaData('data/', end_date = '2022-09-30')
SMB_proxy = fd.get_size_data()['Hi 10'] - fd.get_size_data()['Lo 10']
HML_proxy = fd.get_book_to_market_data()['Hi 10'] - fd.get_book_to_market_data()['Lo 10']

# Fama-French 5-factor model components
RMW_proxy = fd.get_operating_profitability_data()['Hi 10'] - fd.get_operating_profitability_data()['Lo 10']
CMA_proxy = fd.get_investment_data()['Hi 10'] - fd.get_investment_data()['Lo 10']

# And momentum
MOM_proxy = fd.get_momentum_data()['Hi PRIOR'] - fd.get_momentum_data()['Lo PRIOR']

SMB_proxy.name = 'SMB'
HML_proxy.name = 'HML'
RMW_proxy.name = 'RMW'
CMA_proxy.name = 'CMA'
MOM_proxy.name = 'MOM'

In [16]:
fama_factors = pd.concat([SMB_proxy.reset_index(drop=True), HML_proxy.reset_index(drop=True), RMW_proxy.reset_index(drop=True), 
                        CMA_proxy.reset_index(drop=True), MOM_proxy.reset_index(drop=True)], axis = 1)

df = pd.concat([df.reset_index(drop=True), fama_factors.reset_index(drop=True)], axis = 1)
df.head(3)

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks,SMB,HML,RMW,CMA,MOM
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,-0.115244,0.310362,-0.056372,0.146697,0.420152,0.55,-2.66,2.42,3.07,2.98
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,0.163655,0.445749,0.254621,0.827600,1.035070,2.67,1.67,0.54,2.32,3.23
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.161331,0.375940,0.036411,-0.987600,-0.225803,-0.73,-1.77,2.05,-1.02,1.16


In [17]:
## We don't want to include MOM if we are basing this on the momentum portfolios right?

regressors=regressors.append(pd.Index(['SMB','HML','RMW','CMA']))
regressors

Index(['exmt', 'share_issue_delta', 'shortterm_rev_delta',
       'longterm_rev_delta', 'innov_delta', 'growth_delta', 'durable_delta',
       'service_minus_goods', 'needs_minus_wants', 'accrual_delta',
       'cashflow_price_delta', 'market_beta_delta', 'dividend_yield_delta',
       'earnings_price_delta', 'variance_delta', 'AgricMinusSteel',
       'AgricMinusChips', 'SodaMinusTxtls', 'HshldMinusClths',
       'HshldMinusTxtls', 'HshldMinusAero', 'HshldMinusBoxes',
       'HshldMinusBanks', 'MedEqMinusChips', 'DrugsMinusHardw',
       'DrugsMinusBanks', 'SMB', 'HML', 'RMW', 'CMA', 'SMB', 'HML', 'RMW',
       'CMA'],
      dtype='object')

In [24]:
smf.quantreg('spread ~ 1 + ' + ' + '.join(regressors), data=df).fit(q=.10).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         QuantReg Regression Results                          
==============================================================================
Dep. Variable:                 spread   Pseudo R-squared:               0.2210
Model:                       QuantReg   Bandwidth:                       1.634
Method:                 Least Squares   Sparsity:                        21.06
Date:                Wed, 12 Apr 2023   No. Observations:                  711
Time:                        22:08:24   Df Residuals:                      680
                                        Df Model:                           30
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -3.7405      0.290    -12.908      0.000      -4.310      -3.172
exmt                     7.7821     10.879      0.715      0.475     -13.578      29.142
share_issue_delta        0.1801      0.127      1.416      0.157      -0.070       0.430
shortterm_rev_delta      0.1433      0.070      2.054      0.040       0.006       0.280
longterm_rev_delta       0.0477      0.087      0.550      0.582      -0.122       0.218
innov_delta              0.3400      0.162      2.103      0.036       0.022       0.657
growth_delta            -0.1861      0.184     -1.010      0.313      -0.548       0.176
durable_delta           -0.0643      0.082     -0.788      0.431      -0.225       0.096
service_minus_goods     -0.0070      0.010     -0.693      0.489      -0.027       0.013
needs_minus_wants       -0.0067      0.010     -0.677      0.498      -0.026       0.013
accrual_delta           -0.2285      0.116     -1.968      0.050      -0.456      -0.000
cashflow_price_delta     0.0259      0.153      0.170      0.865      -0.275       0.326
market_beta_delta       -0.0148      0.113     -0.131      0.896      -0.236       0.206
dividend_yield_delta    -0.3538      0.100     -3.529      0.000      -0.551      -0.157
earnings_price_delta     0.0433      0.143      0.303      0.762      -0.237       0.324
variance_delta          -0.3694      0.113     -3.258      0.001      -0.592      -0.147
AgricMinusSteel          0.9609      0.458      2.097      0.036       0.061       1.861
AgricMinusChips         -0.0928      0.567     -0.164      0.870      -1.207       1.021
SodaMinusTxtls           0.3192      0.457      0.698      0.485      -0.578       1.216
HshldMinusClths          0.5033      0.403      1.250      0.212      -0.288       1.294
HshldMinusTxtls         -0.0696      0.679     -0.102      0.918      -1.403       1.264
HshldMinusAero          -0.0104      0.402     -0.026      0.979      -0.799       0.779
HshldMinusBoxes          0.0550      0.361      0.152      0.879      -0.655       0.765
HshldMinusBanks          0.6630      0.840      0.790      0.430      -0.986       2.312
MedEqMinusChips         -0.0026      0.491     -0.005      0.996      -0.966       0.961
DrugsMinusHardw          0.8622      0.752      1.147      0.252      -0.614       2.338
DrugsMinusBanks         -0.1611      0.823     -0.196      0.845      -1.777       1.455
SMB                     -0.2889      0.115     -2.505      0.012      -0.515      -0.062
HML                     -0.0434      0.115     -0.377      0.706      -0.269       0.183
RMW                     -0.1340      0.126     -1.061      0.289      -0.382       0.114
CMA                     -0.3671      0.140     -2.628      0.009      -0.641      -0.093
========================================================================================

The condition number is large, 1.15e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""